# <center> RCA$^{++}$ on simulated data </center>

In [1]:
import numpy as np
import os
from astropy.io import fits
import matplotlib.pyplot as plt
import rca

Define a function for visual inspection:

In [2]:
def plot_func(im, cmap='gist_stern', title=''):
    plt.imshow(im, cmap=cmap, interpolation='Nearest')
    plt.title(title)
    plt.colorbar()
    plt.xticks([])
    plt.yticks([])
    plt.show()
    plt.close()

# Prepare data

Load data; `RCADECONV_data.fits` is splitted into 2 hdu, the first one contains 625 simulated observed stars and their positions and the second one contains 625 simulated observed galaxies and their positions.`RCADECONV_groundtruth.fits` is also splitted into 2 hdu, the first one contains 625 simulated PSFs and their positions and the second one contains 625 simulated true galaxies and their positions.
All the data are simulated at the same positions.

In [3]:
hdulist1 = fits.open('RCADECONV_data.fits')
hdulist2 = fits.open('RCADECONV_groundtruth.fits')

In [4]:
hdu1_1 = hdulist1[0]
hdu1_2 = hdulist1[1]
hdu2_1 = hdulist2[0]
hdu2_2 = hdulist2[1]

For each hdu, retrieve the data and their related positons in arrays:

In [5]:
def separate(hdu):
    obs, obs_pos = [], []
    for m in range(25):
        for n in range(25):
            obs.append([[hdu.data[51*m+i][51*n+j] for i in range(51)] for j in range(51)])
            obs_pos.append([51*m+12, 51*n+12])
    return np.array(obs), np.array(obs_pos)

In [6]:
galaxies, galaxies_pos = separate(hdu1_1)
stars, stars_pos = separate(hdu1_2)
galaxies_truth, _ = separate(hdu2_1)
PSFs_truth, _ = separate(hdu2_2)

Split them into stars and 'train' and 'test' galaxies:

In [7]:
gal_idx = np.random.choice(galaxies.shape[0], 156 , False)
star_idx = np.random.choice(list(set(range(galaxies.shape[0])) - set(gal_idx)),313,False)
test_idx = list(set(range(galaxies.shape[0])) - set(gal_idx)- set(star_idx))

In [8]:
train_galaxies, train_galaxies_pos = galaxies[gal_idx], galaxies_pos[gal_idx]
test_galaxies, test_galaxies_pos = galaxies[test_idx], galaxies_pos[test_idx]
stars, stars_pos = stars[star_idx], stars_pos[star_idx]

Retrieve the ground truth PSFs at the positions of the 'test' galaxies set to compare the PSFs estimate model:

In [9]:
test_PSFs_truth = PSFs_truth[test_idx]

# Run RCA$^{++}$

RCA$^{++}$ needs data to be in $(p\times p\times n_\rm{stars})$ format:

In [10]:
stars = rca.utils.rca_format(stars)
train_galaxies = rca.utils.rca_format(train_galaxies)

RCA$^{++}$ expects at least one parameter: `n_comp` is the number of eigenPSFs.

In [11]:
n_comp = 4

In [12]:
rca_runner = rca.RCA(n_comp)

In [ ]:
S, A = rca_runner.fit(stars, train_galaxies, stars_pos, train_galaxies_pos)

the shifts will be estimated from the data using the default Gaussian
window of 7.5 pixels.
Running basic initialization tasks...


/Users/charles/anaconda2/lib/python2.7/site-packages/modopt/signal/wavelet.py:160: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  fake_data[list(zip(data_shape // 2))] = 1


... Done.
Constructing graph constraint...
 > power max =  4.899635092318147
 > selected e: 0.0199024747603	selected a: 0.613757535399	 chosen index: 4/5
 > selected e: 1.23694267422	selected a: 0.613757535399	 chosen index: 1/5
 > selected e: 1.23694267422	selected a: 0.340854253267	 chosen index: 0/5
 > selected e: 1.23694267422	selected a: 0.613757535399	 chosen index: 2/5
... Done.


/Users/charles/anaconda2/lib/python2.7/site-packages/modopt/base/types.py:137: FutureWarning: Conversion of the second argument of issubdtype from `'float'` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype('float').type`.
  (not np.issubdtype(val.dtype, dtype))):
N/A% (0 of 200) |                        | Elapsed Time: 0:00:00 ETA:  --:--:--

 - ITERATION: 1
 - COST: 2450.524855175574



...

 - COST: 285.25338413606363

 - ITERATION: 96


 62% (94 of 150) |##############         | Elapsed Time: 0:05:01 ETA:   0:02:13

 - DATA FIDELITY (X): 253.46234439956524


 62% (94 of 150) |##############         | Elapsed Time: 0:05:02 ETA:   0:02:52

 - Min (X): 1.696470847509749e-65
 - L1 NORM (X): 31.643232311776714


In [ ]:
rca_runner.quicksave('simulated_data_fitted_model')

To compare estimate PSFs with true PSFs, we need their shifts:

In [ ]:
cents = []
for psf in test_PSFs_truth:
    cents += [rca.utils.CentroidEstimator(psf)]
test_shifts = np.array([ce.return_shifts() for ce in cents])

Estimate PSF all the positions. `shifts` are set to get PSFs estimate sampled on the right grid for comparisons with true PSFs.

In [ ]:
test_PSFs_estimate = rca_runner.estimate_psf(test_galaxies_pos, apply_degradation=True, shifts=test_shifts)

Normalize all the PSFs for comparisons:

In [ ]:
for j in range(test_PSFs_estimate.shape[0]):
    test_PSFs_estimate[j] /= test_PSFs_estimate[j].sum()
    test_PSFs_truth[j] /= test_PSFs_truth[j].sum()

## EigenPSFs

In [ ]:
eigenPSFs = rca.utils.reg_format(S)
for epsf in eigenPSFs:
    plot_func(abs(epsf))

## Few examples

In [ ]:
for j in np.random.choice(test_galaxies.shape[0], 3, False):
    true_psf = test_PSFs_truth[j]
    plot_func(true_psf, title='True PSF')
    est_psf = test_PSFs_estimate[j]
    plot_func(est_psf, title=r'$RCA^{++}$ PSF model')
    plot_func(np.abs(true_psf-est_psf), title='residual')
    print j